<a href="https://colab.research.google.com/github/yogakun01/UTS-SistemPakar/blob/main/SISTEM_PAKAR_DIAGNOSA_PENYAKIT_TANAMAN_PADI_DENGAN_CERTAINTY_FACTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SISTEM PAKAR DIAGNOSA PENYAKIT TANAMAN PADI
# Menggunakan Certainty Factor (CF) dan Forward Chaining
# Ujian Tengah Semester - Sistem Pakar
# Nama: [I Made Sedana Yoga]
# NIM: [2301020063]

In [1]:
import os

In [2]:
# 1. KNOWLEDGE BASE - DEFINISI PENYAKIT, GEJALA, DAN RULES

# Database Penyakit
PENYAKIT = {
    'P1': {
        'nama': 'Hawar Daun Bakteri',
        'deskripsi': 'Penyakit yang disebabkan oleh bakteri Xanthomonas oryzae',
        'penanganan': [
            '- Gunakan varietas tahan penyakit',
            '- Sanitasi lahan dan pemangkasan daun terinfeksi',
            '- Aplikasi bakterisida berbahan tembaga',
            '- Atur jarak tanam yang baik untuk sirkulasi udara'
        ]
    },
    'P2': {
        'nama': 'Blas',
        'deskripsi': 'Penyakit yang disebabkan oleh jamur Pyricularia oryzae',
        'penanganan': [
            '- Gunakan varietas tahan blas',
            '- Kurangi penggunaan nitrogen berlebih',
            '- Aplikasi fungisida (seperti Tricyclazole)',
            '- Jaga kelembaban tidak terlalu tinggi'
        ]
    },
    'P3': {
        'nama': 'Busuk Batang',
        'deskripsi': 'Penyakit yang disebabkan oleh bakteri atau jamur pada batang',
        'penanganan': [
            '- Perbaiki drainase lahan',
            '- Buang tanaman yang terinfeksi',
            '- Aplikasi fungisida/bakterisida sistemik',
            '- Hindari pemupukan nitrogen berlebihan'
        ]
    },
    'P4': {
        'nama': 'Tungro',
        'deskripsi': 'Penyakit virus yang ditularkan oleh wereng hijau',
        'penanganan': [
            '- Kendalikan populasi wereng hijau (vektor)',
            '- Gunakan varietas tahan tungro',
            '- Cabut dan musnahkan tanaman terinfeksi',
            '- Aplikasi insektisida untuk wereng'
        ]
    }
}

# Database Gejala
GEJALA = {
    'G1': 'Daun menguning',
    'G2': 'Bercak coklat pada daun',
    'G3': 'Daun mengering',
    'G4': 'Batang busuk dan berbau',
    'G5': 'Pertumbuhan terhambat',
    'G6': 'Daun berlubang',
    'G7': 'Daun berkerut',
    'G8': 'Tanaman kerdil'
}

# Rules dengan CF Pakar (Forward Chaining)
# Format: (kode_penyakit, kode_gejala, CF_pakar)
RULES = [
    # Hawar Daun Bakteri (P1)
    ('P1', 'G1', 0.6),
    ('P1', 'G2', 0.8),
    ('P1', 'G6', 0.7),

    # Blas (P2)
    ('P2', 'G2', 0.7),
    ('P2', 'G3', 0.8),
    ('P2', 'G5', 0.6),

    # Busuk Batang (P3)
    ('P3', 'G4', 0.9),
    ('P3', 'G5', 0.7),
    ('P3', 'G1', 0.5),

    # Tungro (P4)
    ('P4', 'G1', 0.8),
    ('P4', 'G7', 0.9),
    ('P4', 'G8', 0.85),
]

In [3]:
# 2. FUNGSI PERHITUNGAN CERTAINTY FACTOR

def hitung_cf_gejala(cf_user, cf_pakar):
    """
    Menghitung CF untuk satu gejala
    CF(H,E) = CF(user) × CF(pakar)

    Args:
        cf_user: Certainty factor dari user (0-1)
        cf_pakar: Certainty factor dari pakar/rule (0-1)

    Returns:
        CF hasil perkalian
    """
    return cf_user * cf_pakar

def kombinasi_cf(cf_old, cf_new):
    """
    Menggabungkan CF dari multiple evidence
    CF_kombinasi = CF_old + CF_new × (1 - CF_old)

    Args:
        cf_old: CF yang sudah ada
        cf_new: CF baru yang akan dikombinasikan

    Returns:
        CF hasil kombinasi
    """
    return cf_old + cf_new * (1 - cf_old)

def validasi_cf_input(nilai):
    """Validasi input CF dari user (harus 0-1)"""
    try:
        cf = float(nilai)
        if 0 <= cf <= 1:
            return cf
        else:
            return None
    except:
        return None


In [4]:
# 3. FORWARD CHAINING ENGINE

def forward_chaining(gejala_user):
    """
    Proses forward chaining untuk menghitung CF setiap penyakit

    Args:
        gejala_user: Dictionary {kode_gejala: cf_user}

    Returns:
        Dictionary {kode_penyakit: cf_total}
    """
    # Inisialisasi CF untuk setiap penyakit
    cf_penyakit = {kode: 0 for kode in PENYAKIT.keys()}

    # Dictionary untuk menyimpan CF per gejala untuk setiap penyakit
    cf_detail = {kode: [] for kode in PENYAKIT.keys()}

    print("\n" + "="*70)
    print("PROSES FORWARD CHAINING")
    print("="*70)

    # Iterasi melalui semua rules
    for penyakit_kode, gejala_kode, cf_pakar in RULES:
        # Cek apakah gejala ini dipilih user
        if gejala_kode in gejala_user:
            cf_user = gejala_user[gejala_kode]

            # Hitung CF untuk gejala ini
            cf_gejala = hitung_cf_gejala(cf_user, cf_pakar)

            print(f"\nRule: IF {gejala_kode} ({GEJALA[gejala_kode]}) THEN {penyakit_kode} ({PENYAKIT[penyakit_kode]['nama']})")
            print(f"  CF(user) = {cf_user:.2f}")
            print(f"  CF(pakar) = {cf_pakar:.2f}")
            print(f"  CF(H,E) = {cf_user:.2f} × {cf_pakar:.2f} = {cf_gejala:.4f}")

            # Simpan detail
            cf_detail[penyakit_kode].append({
                'gejala': gejala_kode,
                'cf_user': cf_user,
                'cf_pakar': cf_pakar,
                'cf_hasil': cf_gejala
            })

            # Kombinasikan dengan CF yang sudah ada
            cf_old = cf_penyakit[penyakit_kode]
            if cf_old == 0:
                cf_penyakit[penyakit_kode] = cf_gejala
                print(f"  CF({penyakit_kode}) = {cf_gejala:.4f} (first evidence)")
            else:
                cf_new = kombinasi_cf(cf_old, cf_gejala)
                print(f"  CF_kombinasi = {cf_old:.4f} + {cf_gejala:.4f} × (1 - {cf_old:.4f})")
                print(f"  CF_kombinasi = {cf_old:.4f} + {cf_gejala * (1 - cf_old):.4f} = {cf_new:.4f}")
                cf_penyakit[penyakit_kode] = cf_new

    return cf_penyakit, cf_detail

In [5]:
# 4. USER INTERFACE (CLI)

def clear_screen():
    """Clear screen untuk berbagai OS"""
    os.system('cls' if os.name == 'nt' else 'clear')

def tampilkan_header():
    """Menampilkan header aplikasi"""
    print("="*70)
    print(" " * 10 + "SISTEM PAKAR DIAGNOSA PENYAKIT TANAMAN PADI")
    print(" " * 15 + "Menggunakan Certainty Factor (CF)")
    print("="*70)

def tampilkan_daftar_gejala():
    """Menampilkan daftar gejala yang tersedia"""
    print("\nDAFTAR GEJALA PENYAKIT TANAMAN PADI:")
    print("-" * 70)
    for kode, nama in GEJALA.items():
        print(f"  {kode}: {nama}")
    print("-" * 70)

def input_gejala():
    """Input gejala dan CF dari user"""
    gejala_user = {}

    tampilkan_daftar_gejala()

    print("\nPetunjuk:")
    print("- Masukkan kode gejala yang dialami tanaman (contoh: G1)")
    print("- Masukkan CF (tingkat keyakinan) untuk setiap gejala (0-1)")
    print("  * 0.0 - 0.2 = Tidak yakin")
    print("  * 0.3 - 0.5 = Kurang yakin")
    print("  * 0.6 - 0.8 = Cukup yakin")
    print("  * 0.9 - 1.0 = Sangat yakin")
    print("- Ketik 'selesai' jika sudah selesai input gejala\n")

    while True:
        kode_gejala = input("Masukkan kode gejala (atau 'selesai'): ").strip().upper()

        if kode_gejala == 'SELESAI':
            break

        if kode_gejala not in GEJALA:
            print("❌ Kode gejala tidak valid! Silakan coba lagi.")
            continue

        if kode_gejala in gejala_user:
            print("⚠ Gejala sudah diinput sebelumnya!")
            continue

        while True:
            cf_input = input(f"Masukkan CF untuk '{GEJALA[kode_gejala]}' (0-1): ").strip()
            cf = validasi_cf_input(cf_input)

            if cf is not None:
                gejala_user[kode_gejala] = cf
                print(f"✓ Gejala {kode_gejala} dengan CF={cf:.2f} berhasil ditambahkan\n")
                break
            else:
                print("❌ CF harus berupa angka antara 0 dan 1! Silakan coba lagi.")

    return gejala_user

def tampilkan_hasil(cf_penyakit, gejala_user):
    """Menampilkan hasil diagnosa dengan ranking"""
    print("\n" + "="*70)
    print("HASIL DIAGNOSA")
    print("="*70)

    # Filter penyakit dengan CF > 0
    hasil = [(kode, cf) for kode, cf in cf_penyakit.items() if cf > 0]

    if not hasil:
        print("\n❌ Tidak ada penyakit yang teridentifikasi.")
        print("   Gejala yang diinput tidak cukup untuk mendiagnosa penyakit.")
        return

    # Sort berdasarkan CF (descending)
    hasil.sort(key=lambda x: x[1], reverse=True)

    print(f"\nGejala yang diinput: {len(gejala_user)} gejala")
    print(f"Penyakit teridentifikasi: {len(hasil)} penyakit\n")

    # Tampilkan ranking
    print(f"{'Rank':<6} {'Kode':<6} {'Nama Penyakit':<25} {'CF':<10} {'Persentase':<12}")
    print("-" * 70)

    for rank, (kode, cf) in enumerate(hasil, 1):
        nama = PENYAKIT[kode]['nama']
        persentase = cf * 100
        print(f"{rank:<6} {kode:<6} {nama:<25} {cf:<10.4f} {persentase:<12.2f}%")

    # Diagnosa utama
    penyakit_utama = hasil[0][0]
    cf_utama = hasil[0][1]

    print("\n" + "="*70)
    print("KESIMPULAN DIAGNOSA")
    print("="*70)

    if cf_utama >= 0.8:
        tingkat = "SANGAT TINGGI"
        emoji = "🔴"
    elif cf_utama >= 0.6:
        tingkat = "TINGGI"
        emoji = "🟠"
    elif cf_utama >= 0.4:
        tingkat = "SEDANG"
        emoji = "🟡"
    else:
        tingkat = "RENDAH"
        emoji = "🟢"

    print(f"\n{emoji} PENYAKIT: {PENYAKIT[penyakit_utama]['nama']}")
    print(f"   Certainty Factor: {cf_utama:.4f} ({cf_utama*100:.2f}%)")
    print(f"   Tingkat Keyakinan: {tingkat}")
    print(f"\n   Deskripsi: {PENYAKIT[penyakit_utama]['deskripsi']}")

    print(f"\n{'='*70}")
    print("REKOMENDASI PENANGANAN")
    print("="*70)
    for penanganan in PENYAKIT[penyakit_utama]['penanganan']:
        print(f"  {penanganan}")

    # Jika ada penyakit lain dengan CF signifikan
    if len(hasil) > 1 and hasil[1][1] >= 0.3:
        print(f"\n⚠ PERHATIAN: Terdapat kemungkinan penyakit lain:")
        for kode, cf in hasil[1:]:
            if cf >= 0.3:
                print(f"   - {PENYAKIT[kode]['nama']} (CF: {cf:.4f} / {cf*100:.2f}%)")


In [6]:
# 5. FUNGSI TESTING OTOMATIS

def run_test_case(test_name, gejala_input):
    """Menjalankan satu test case"""
    print("\n" + "="*70)
    print(f"TEST CASE: {test_name}")
    print("="*70)

    print("\nGejala Input:")
    for kode, cf in gejala_input.items():
        print(f"  {kode} ({GEJALA[kode]}): CF = {cf:.2f}")

    # Proses diagnosa
    cf_penyakit, cf_detail = forward_chaining(gejala_input)

    # Tampilkan hasil
    tampilkan_hasil(cf_penyakit, gejala_input)

def run_all_tests():
    """Menjalankan semua test case"""
    print("\n" + "="*70)
    print("MENJALANKAN TEST CASES")
    print("="*70)

    # Test Case 1: Hawar Daun Bakteri dominan
    test_case_1 = {
        'G1': 0.7,  # Daun menguning
        'G2': 0.9,  # Bercak coklat
        'G6': 0.8   # Daun berlubang
    }
    run_test_case("Hawar Daun Bakteri (Dominan)", test_case_1)

    # Test Case 2: Tungro dominan
    test_case_2 = {
        'G1': 0.9,  # Daun menguning
        'G7': 0.95, # Daun berkerut
        'G8': 0.9   # Tanaman kerdil
    }
    run_test_case("Tungro (Dominan)", test_case_2)

    # Test Case 3: Mixed symptoms (Blas dan Busuk Batang)
    test_case_3 = {
        'G1': 0.6,  # Daun menguning
        'G2': 0.7,  # Bercak coklat
        'G3': 0.5,  # Daun mengering
        'G4': 0.8,  # Batang busuk
        'G5': 0.7   # Pertumbuhan terhambat
    }
    run_test_case("Mixed Symptoms (Multiple Diseases)", test_case_3)

    print("\n" + "="*70)
    print("SEMUA TEST CASE SELESAI")
    print("="*70)

In [ ]:
# 6. MAIN PROGRAM

def main():
    """Program utama"""
    while True:
        clear_screen()
        tampilkan_header()

        print("\nMENU:")
        print("1. Diagnosa Penyakit (Input Manual)")
        print("2. Jalankan Test Cases (Otomatis)")
        print("3. Lihat Knowledge Base")
        print("4. Keluar")

        pilihan = input("\nPilih menu (1-4): ").strip()

        if pilihan == '1':
            # Mode input manual
            print("\n" + "="*70)
            print("MODE DIAGNOSA MANUAL")
            print("="*70)

            gejala_user = input_gejala()

            if not gejala_user:
                print("\n❌ Tidak ada gejala yang diinput!")
                input("\nTekan Enter untuk kembali ke menu...")
                continue

            # Proses forward chaining
            cf_penyakit, cf_detail = forward_chaining(gejala_user)

            # Tampilkan hasil
            tampilkan_hasil(cf_penyakit, gejala_user)

            input("\nTekan Enter untuk kembali ke menu...")

        elif pilihan == '2':
            # Mode testing otomatis
            run_all_tests()
            input("\nTekan Enter untuk kembali ke menu...")

        elif pilihan == '3':
            # Lihat knowledge base
            print("\n" + "="*70)
            print("KNOWLEDGE BASE")
            print("="*70)

            print("\nDAFTAR PENYAKIT:")
            for kode, data in PENYAKIT.items():
                print(f"\n{kode}: {data['nama']}")
                print(f"   {data['deskripsi']}")

            print("\n" + "-"*70)
            tampilkan_daftar_gejala()

            print("\nDAFTAR RULES:")
            print("-" * 70)
            for penyakit_kode, gejala_kode, cf in RULES:
                print(f"IF {gejala_kode} ({GEJALA[gejala_kode]}) THEN {penyakit_kode} ({PENYAKIT[penyakit_kode]['nama']}) [CF={cf}]")

            input("\nTekan Enter untuk kembali ke menu...")

        elif pilihan == '4':
            print("\n" + "="*70)
            print("Terima kasih telah menggunakan Sistem Pakar!")
            print("="*70)
            break

        else:
            print("\n❌ Pilihan tidak valid!")
            input("Tekan Enter untuk kembali ke menu...")

# Jalankan program
if __name__ == "__main__":
    main()

          SISTEM PAKAR DIAGNOSA PENYAKIT TANAMAN PADI
               Menggunakan Certainty Factor (CF)

MENU:
1. Diagnosa Penyakit (Input Manual)
2. Jalankan Test Cases (Otomatis)
3. Lihat Knowledge Base
4. Keluar

Pilih menu (1-4): 1

MODE DIAGNOSA MANUAL

DAFTAR GEJALA PENYAKIT TANAMAN PADI:
----------------------------------------------------------------------
  G1: Daun menguning
  G2: Bercak coklat pada daun
  G3: Daun mengering
  G4: Batang busuk dan berbau
  G5: Pertumbuhan terhambat
  G6: Daun berlubang
  G7: Daun berkerut
  G8: Tanaman kerdil
----------------------------------------------------------------------

Petunjuk:
- Masukkan kode gejala yang dialami tanaman (contoh: G1)
- Masukkan CF (tingkat keyakinan) untuk setiap gejala (0-1)
  * 0.0 - 0.2 = Tidak yakin
  * 0.3 - 0.5 = Kurang yakin
  * 0.6 - 0.8 = Cukup yakin
  * 0.9 - 1.0 = Sangat yakin
- Ketik 'selesai' jika sudah selesai input gejala

Masukkan kode gejala (atau 'selesai'): G1
Masukkan CF untuk 'Daun menguning' (